# 深層学習ノートブック-3 pytorchによる多項ロジスティック回帰
参考：  
* [行列積](https://w3e.kanazawa-it.ac.jp/math/category/gyouretu/senkeidaisu/henkan-tex.cgi?target=/math/category/gyouretu/senkeidaisu/gyouretu-no-seki.html)

pytorchのAutograd, Tensorを用いて多項ロジスティック回帰による分類をスクラッチで実装する。  
ここで実装する一連の流れは深層学習におけるモデル学習プロセスの基礎であり、非常に重要。  

ここではMNIST（手書き数字(0~9)）データセットを用いた10クラス分類のタスクを多項ロジスティック回帰で解くことを考える。  
MNISTデータセットにある各pixel値(8x8=64)を特徴量として扱う。  

データ数をm、特徴量の行列を$\bm{X}$、$\bm{X}$に対応する重み行列を$W^T$※、バイアス項の行列を$\bm{b}$とすると、  
softmax関数への入力$\bm{z}$は下記のように表せる。（pytorchではこの形を採用している。）  
※行列積の形で表わすために転置をとっている。


$\bm{z} = \bm{XW}^T+\bm{b} $  
$=\left(
\begin{matrix} 
x_{1,1} & x_{1,2} & ... & x_{1,64}\\ 
x_{2,1} & x_{2,2} & ... & x_{2, 64}\\
. & . & ... & .\\
x_{m,1} & x_{m,2} & ... & x_{m, 64}
\end{matrix} 
\right)$
$\left(
\begin{matrix} 
w_{1,1} & w_{1,2} & ... & w_{1,10}\\ 
w_{2,1} & w_{2,2} & ... & x_{2,10}\\
. & . & ... & .\\
w_{64,1} & x_{64,2} & ... & x_{64, 10}
\end{matrix} 
\right)$
$+\left(
\begin{matrix} 
b_{1} & b_{2} & ... & b_{10}\\ 
b_{1} & b_{2} & ... & b_{10}\\
. & . & ... & .\\
b_{1} & b_{2} & ... & b_{10}
\end{matrix}
\right)$  

$=\left(
\begin{matrix} 
\sum_{k=1}^{64} x_{1, k}w_{k, 1}+b_1 & \sum_{k=1}^{64} x_{1, k}w_{k, 2}+b_2 & ... & \sum_{k=1}^{64} x_{1, k}w_{k, 10}+b_{10}\\ 
\sum_{k=1}^{64} x_{2, k}w_{k, 1}+b_1 & \sum_{k=1}^{64} x_{2, k}w_{k, 2}+b_2 & ... & \sum_{k=1}^{64} x_{2, k}w_{k, 10}+b_{10}\\ 
. & . & ... & .\\
\sum_{k=1}^{64} x_{m, k}w_{k, 1}+b_1 & \sum_{k=1}^{64} x_{m, k}w_{k, 2}+b_2 & ... & \sum_{k=1}^{64} x_{m, k}w_{k, 10}+b_{10}\\ 
\end{matrix} 
\right)$

$=\left(
\begin{matrix} 
z_{1,1} & z_{1,2} & ... & z_{1,10}\\ 
z_{2,1} & z_{2,2} & ... & z_{2,10}\\
. & . & ... & .\\
z_{m,1} & z_{m,2} & ... & z_{m, 10}
\end{matrix} 
\right)$

$\bm{X}$のshapeはデータ数m × 特徴量数64  
$\bm{W}^T$は重みづけの対象パラメータ数（すなわち特徴量数64）× 最終的な出力列数（10クラス分類なので10）※、  
$\bm{b}$は1 × 最終的な出力列数となるが、上記ではbroadcastされた状態で書いている。  
最終的に$\bm{z}$は各データ数 × クラス数というshapeで表わされ、クラスごとの線形回帰の結果(をsoftmaxに入れたもの)が各要素に対応する。  

※転置前の$\bm{W}$は10*64

# 大まかな流れ
1. データロード
2. 前処理
    * 目的変数のエンコーディング： torch.nn.functional.one_hot()
    * ピクセル値の標準化
3. パラメータ初期化
    * torch.rand(requires_grad=True)
4. 損失関数とsoftmax関数実装
5. for文で学習ループ作成（５回）。※このループの単位をepochという  
6. 入力データ$\bm{X}$および教師ラベル$\bm{Y}$作成
7. 出力結果$\bm{Z}$計算
8. softmaxで予測値計算
9. 損失計算
10. 勾配計算
11. パラメータ更新
12. 勾配初期化
    * .grad.zero_()を用いる。  
    ※ちなみにpytorchの中で最後にアンダースコアが入るメソッドは読み出し元のインスタンスの属性に値を代入することを示す。  
    この場合は.grad.zero_()を読みだしたtensorのgradにゼロが代入される。
13. 損失ログ出力
    

# 前処理・初期化

In [64]:
import sklearn
import torch
import torch.nn.functional as F  #pytorchの便利関数はFでimportすることが多い。

In [218]:
# 変数定義
learning_rate = 0.03
loss_log = []  #損失記録用のリスト

In [235]:
# データロード
dataset = sklearn.datasets.load_digits()
feature_names = dataset['feature_names']
X = torch.tensor(dataset['data'], dtype=torch.float32)
y_true = torch.tensor(dataset['target'])

# shape確認
print(f'shape of X: {X.shape}')
print(X[1])
print('==========================')
print(f'shape of Y: {Y.shape}')
print(Y[1])

shape of X: torch.Size([1797, 64])
tensor([ 0.,  0.,  0., 12., 13.,  5.,  0.,  0.,  0.,  0.,  0., 11., 16.,  9.,
         0.,  0.,  0.,  0.,  3., 15., 16.,  6.,  0.,  0.,  0.,  7., 15., 16.,
        16.,  2.,  0.,  0.,  0.,  0.,  1., 16., 16.,  3.,  0.,  0.,  0.,  0.,
         1., 16., 16.,  6.,  0.,  0.,  0.,  0.,  1., 16., 16.,  6.,  0.,  0.,
         0.,  0.,  0., 11., 16., 10.,  0.,  0.])
shape of Y: torch.Size([1797, 10])
tensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 0])


datasetのimagesは1797 x 8 x 8の形式になっており、reshapeする必要があるため、'data'から読み込んだ方が楽。

In [246]:
# 目的変数のエンコーディング
y_true = F.one_hot(y_true, num_classes=10)
print(f'shape of Y: {y_true.shape}')
print(y_true)

shape of Y: torch.Size([1797, 10])
tensor([[1, 0, 0,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 1, 0]])


In [221]:
# 学習データの標準化（ピクセル値を標準化）
for i in range(X.shape[1]):
    mean = X[:, i].mean()
    std = X[:, i].std()
    if(std == 0.):
        X[:, i] = 0.
    else:
        X[:, i] = (X[:, i] - mean) / std


pytorchには標準で標準化のためのクラスがないので自前で実装する必要あり。  
※同じようなスケールのピクセル値の列なので、全列まとめて平均とって標準偏差で割るでも大差ないかも。

In [90]:
# 重みW^T、バイアス項bの初期化
W = torch.rand(size=(10, 64) ,requires_grad=True) #出力×入力
b = torch.rand(size=(1, 10), requires_grad=True) # 1 x 出力

print(W.shape)
print(W[1])
print()
print(b)

torch.Size([10, 64])
tensor([0.8464, 0.9092, 0.3408, 0.4982, 0.5163, 0.9932, 0.5947, 0.5541, 0.0591,
        0.3156, 0.8722, 0.9538, 0.3694, 0.3806, 0.9416, 0.4064, 0.9667, 0.9310,
        0.9185, 0.4682, 0.8012, 0.6893, 0.2805, 0.0374, 0.9744, 0.3576, 0.9610,
        0.6810, 0.3514, 0.9300, 0.6123, 0.3133, 0.1097, 0.3547, 0.0034, 0.9448,
        0.9941, 0.0708, 0.4644, 0.0984, 0.3339, 0.3055, 0.3337, 0.6091, 0.4175,
        0.5785, 0.8451, 0.1898, 0.0282, 0.9654, 0.8262, 0.2924, 0.2932, 0.8604,
        0.4287, 0.3145, 0.0717, 0.2354, 0.7837, 0.2466, 0.5344, 0.4616, 0.7356,
        0.2248], grad_fn=<SelectBackward0>)

tensor([[0.5165, 0.1129, 0.7141, 0.1706, 0.0847, 0.3603, 0.1588, 0.1910, 0.8762,
         0.1424]], requires_grad=True)


# softmax関数実装

In [214]:
# softmax関数の実装
def softmax_func(X):
    '''
    X: input tensor.行は各データ、列は各クラスを想定。
    '''
    # x_kが大きすぎると、e^xがinfになるのでmax(x_1, x_2,...,x_K)を各x_kから引く。
    # 各データ（各行）について最大値を求める必要があるので、dimにRank1(列方向で比較)を指定する。
    max_val = X.max(dim=1, keepdim=True).values
    # 各要素のe^xを計算（これが分子になる）
    e_x = (X - max_val).exp()

    # softmax関数の分母の計算
    # 各データについて合計したいので、dim=1を設定。また、分母が0になることを防ぐために分母の式に1e-10を足しておく
    denominator = e_x.sum(dim=1, keepdim=True) + 1e-10

    return e_x / denominator


## ※dim=1の補足
列方向や行方向について統計量を求められるので便利だが、デフォルトではshapeが変わるので注意が必要。  
これを防ぐにはkeepdim=Trueに設定するとよい。こうすると次元は元のまま維持される。    
仮に上記関数にWを入れた場合を例に考える。

In [211]:
W.max(dim=1).values.shape

torch.Size([10])

In [212]:
W.max(dim=1, keepdim=True).values.shape

torch.Size([10, 1])

※keepdimを使わずにデフォルトで計算してしまった場合  
max(dim=1)の結果は1次元のtensorであるため、  
W(shape: 10 x 64)とはブロードキャスティングで演算ができない。  
なのでWのRank0に合わせるようにreshapeしてから計算する必要がある。  
分母計算時のsumでも同様。

In [199]:
# 計算不可
W - W.max(dim=1).values

RuntimeError: The size of tensor a (64) must match the size of tensor b (10) at non-singleton dimension 1

In [201]:
# 計算可能
( W - W.max(dim=1).values.reshape((W.shape[0], -1)) )[0]

tensor([-0.8109, -0.9049, -0.9427, -0.9684, -0.1856, -0.9454, -0.5918, -0.0172,
        -0.0462, -0.5656, -0.5253, -0.6007, -0.7508, -0.7995, -0.8733, -0.9322,
        -0.6036, -0.9642, -0.9464, -0.5793, -0.8477, -0.2772, -0.1476, -0.6274,
        -0.3832, -0.6449,  0.0000, -0.4132, -0.8661, -0.7283, -0.2291, -0.2358,
        -0.1362, -0.6353, -0.5935, -0.4778, -0.0388, -0.7224, -0.8076, -0.2210,
        -0.8226, -0.4989, -0.7863, -0.5660, -0.6022, -0.1474, -0.4801, -0.7416,
        -0.8781, -0.5226, -0.2101, -0.1882, -0.4413, -0.2285, -0.9543, -0.6227,
        -0.3525, -0.7373, -0.8028, -0.5557, -0.3733, -0.6344, -0.2418, -0.8341],
       grad_fn=<SelectBackward0>)

# 損失関数（交差エントロピー）実装
$$L(\theta)= - \frac{1}{m}\sum_{i=1}^m \sum_{k=1}^K t_{ik}log(p_k(x_i))$$  

$t_{ik}$はOne-Hot Encodingされた目的変数、$p_k(x_i)$は予測値（ソフトマックス関数の出力）に相当する。  
$p_k(x_i)$が0になるとlog(0)になり、負の無限大に発散するため、  
実装上は1e-10等の小さい値を足しておく。  

In [282]:
# 多クラス分類に対応した交差エントロピー
def cross_entropy(y_true, y_pred):
    '''
    y_true: tensor。One-Hot Encoding済みの正解ラベル。
    y_pred: tensor。予測値。softmax関数の出力(0~100%)。
    '''
    # 損失を計算。最終的な損失はスカラーなので、dimを指定する必要はない。
    return - (y_true * torch.log(y_pred + 1e-10)).sum() / y_true.shape[0]


y_trueは該当するクラス以外ゼロなので、そのまま予測値（log）とのアダマール積を計算すれば損失が計算できる。  